In [44]:
# Function to solve the 0/1 Knapsack problem recursively and return selected items
function knapsack_dq(W, weights, values, n)
    # Base case: no items left or no capacity left
    if n == 0 || W == 0
        return 0, []
    end

    # If the weight of the nth item is more than the knapsack capacity W,
    # then this item cannot be included in the optimal solution
    if weights[n] > W
        return knapsack_dq(W, weights, values, n - 1)
    else
        # Solve the subproblem including the nth item
        value_including_n, items_including_n = knapsack_dq(W - weights[n], weights, values, n - 1)
        value_including_n += values[n]

        # Solve the subproblem excluding the nth item
        value_excluding_n, items_excluding_n = knapsack_dq(W, weights, values, n - 1)

        # Compare the two cases and return the better result
        if value_including_n > value_excluding_n
            return value_including_n, [n; items_including_n]
        else
            return value_excluding_n, items_excluding_n
        end
    end
end




knapsack_dq (generic function with 1 method)

In [7]:
# Function to solve the 0/1 Knapsack problem using dynamic programming
function knapsack_dp(W, weights, values)
    n = length(weights)
    
    # Create a 2D array to store the maximum value at each n, W
    dp = zeros(Int, n + 1, W + 1)

    # Build the dp array bottom-up
    for i in 1:n
        for w in 0:W
            if weights[i] <= w
                dp[i + 1, w + 1] = max(dp[i, w + 1], values[i] + dp[i, w - weights[i] + 1])
            else
                dp[i + 1, w + 1] = dp[i, w + 1]
            end
        end
    end

    # The maximum value is found at dp[n+1, W+1]
    max_value = dp[n + 1, W + 1]

    # Backtrack to find the items included in the optimal solution
    selected_items = []
    w = W
    for i in n:-1:1
        if dp[i + 1, w + 1] != dp[i, w + 1]
            push!(selected_items, i)
            w -= weights[i]
        end
    end

    return max_value, reverse(selected_items)
end

knapsack_dp (generic function with 1 method)

In [79]:
# Example usage
using Random
Random.seed!(10)
weights =rand(1:10, 30) * 10 
values = rand(1:10, 30) * 10
W = 300
n = length(weights)

@time max_value, selected_items = knapsack_dq(W, weights, values, n)
println("\nweights:", weights)
println("Values:", values)
println("\nMaximum value in Knapsack: ", max_value)
println("Selected items: ", reverse(selected_items))

  0.560368 seconds (9.35 M allocations: 504.226 MiB, 7.94% gc time)

weights:[40, 10, 20, 20, 50, 60, 80, 70, 70, 60, 70, 10, 10, 10, 90, 10, 90, 90, 30, 50, 30, 50, 100, 60, 10, 60, 20, 30, 40, 10]
Values:[30, 80, 60, 70, 90, 20, 80, 90, 50, 80, 20, 80, 80, 50, 80, 40, 60, 20, 20, 10, 10, 60, 100, 10, 70, 40, 90, 40, 10, 70]

Maximum value in Knapsack: 930
Selected items: Any[2, 3, 4, 5, 8, 12, 13, 14, 16, 22, 25, 27, 30]


In [80]:
# Example usage
using Random
Random.seed!(10)
weights =rand(1:10, 30) * 10
values = rand(1:10, 30) * 10
W = 300

@time max_value, selected_items = knapsack_dp(W, weights, values)
println("\nweights:", weights)
println("Values:", values)
println("\nMaximum value in Knapsack: ", max_value)
println("Selected items: ", reverse(selected_items))

  0.000049 seconds (13 allocations: 73.781 KiB)

weights:[40, 10, 20, 20, 50, 60, 80, 70, 70, 60, 70, 10, 10, 10, 90, 10, 90, 90, 30, 50, 30, 50, 100, 60, 10, 60, 20, 30, 40, 10]
Values:[30, 80, 60, 70, 90, 20, 80, 90, 50, 80, 20, 80, 80, 50, 80, 40, 60, 20, 20, 10, 10, 60, 100, 10, 70, 40, 90, 40, 10, 70]

Maximum value in Knapsack: 930
Selected items: Any[30, 27, 25, 22, 16, 14, 13, 12, 8, 5, 4, 3, 2]
